# An exploration of tourism during the COVID-19 pandemic

### 7 December 2021

### Megan Mantaro

# Code for Part 1

## Set-up

### Import modules

In [1]:
#Import necessary packages
from pathlib import Path
import pandas as pd
import json
from os import sep
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.legend_handler import HandlerTuple
import matplotlib.dates as mdates
import psycopg2
import datetime as dt
from scipy import stats
from textwrap import fill
from collections import defaultdict

#Set up connection to OxCOVID database
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19',
    user='covid19',
    password='covid19')
cur = conn.cursor()

### Functions

In [2]:
def first_day(countrycode, column, greater_than=0):
    '''
    Takes a countrycode and returns the first day a given column has a value 
    greater than a given number (default value is 0, but greater_than argument 
    takes any number). Returns None if the countrycode can't be located.
    
    Args:
        countrycode (str): 3-letter country code
        column (DataFrame column): column to check for first day of a given 
            value in
        greater_than: value to use in Boolean statement when finding first 
            instance greater than that value; default is 0
    
    Returns:
        DateTime object or None
    '''
    try:
        mask = df_control_epidem_merged[column] > greater_than
        c_grp = df_control_epidem_merged[mask].groupby('countrycode')
        return c_grp['date'].min().loc[countrycode]
    
    except KeyError:
        return None

#Note: The following four functions are essentially the same, with just
#a different filtering level. However, they were created for use in an agg
#function and thus are defined separately to improve readability later in code
def num_days_any_controls(grp):
    '''
    Returns the number of days with any level travel controls
    
    Args:
        grp (GroupBy object): Grouping of countrycode
    
    Returns:
        Number of days where column being aggregated is >= 1
    '''
    return np.sum(grp>=1) 
    
def num_days_2(grp):
    '''
    Returns the number of days with level 2-4 travel controls
    
    Args:
        grp (GroupBy object): Grouping of countrycode
    
    Returns:
        Number of days where column being aggregated is >= 2
    '''
    return np.sum(grp>=2) 
    
def num_days_3(grp):
    '''
    Returns the number of days with level 3-4 travel controls
    
    Args:
        grp (GroupBy object): Grouping of countrycode
    
    Returns:
        Number of days where column being aggregated is >= 3
    '''
    return np.sum(grp>=3)

def num_days_4(grp):
    '''
    Returns the number of days with level 4 travel controls
    
    Args:
        grp (GroupBy object): Grouping of countrycode
    
    Returns:
        Number of days where column being aggregated is >= 4
    '''
    return np.sum(grp==4)

def non_null_controls(countrycode):
    '''
    Takes a countrycode and returns True if a country has any non-null values 
    in the c8_international_travel_controls column of the 
    df_control_epidem_merged dataframe or False if the countrycode 
    only has null values in that column.
    
    Args:
        countrycode (str): 3-letter country code
    
    Returns:
        Boolean value
    '''
    mask = (df_control_epidem_merged['countrycode'] == countrycode)
    by_country = df_control_epidem_merged[mask]
    has_non_null_values = (by_country['c8_international_travel_controls']
                           .notnull().sum() > 0)
    return has_non_null_values

def set_mo_yr_ax(ax, minor_rotation=0, labelsize=12):
    '''
    Formats x-axis with years as major ticks and months as minor ticks. Offers 
    option to rotate the months and change the size of the month/year labels.
    
    Args:
        ax (Matplotlib Axes instance): ax to set x-axis for
        minor_rotation (float): number between 0 and 360; sets month rotation;
            default value is 0 (no rotation)
        labelsize (float): fontsize of month and year labels
    
    Returns:
        Formatted x-axis
    '''
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    for tick in ax.xaxis.get_major_ticks():
        tick.tick1line.set_markersize(0)
        tick.tick2line.set_markersize(0)
        tick.label1.set_horizontalalignment('center')
        tick.set_pad(4.5 * tick.get_pad())
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.remove_overlapping_locs = False
    ax.tick_params(axis="x", 
                   which="minor", 
                   rotation=minor_rotation, 
                   labelsize=labelsize)
    ax.tick_params(axis="x", 
                   which="major", 
                   labelsize=labelsize)

def create_travel_scatter(ax, df, measure, title, kind, trim=False):
    '''
    Plots data against international tourism (as % of GDP) as scatterplot 
    with linear regression model fit (also shows confidence interval, 
    coeff, and p-value). 
    
    Args:
        ax (Matplotlib Axes instance): ax to set x-axis for
        df (Pandas DataFrame instance): DataFrame with data
        measure (Pandas DataFrame column): data to plot along y-axis
        title (str): title of subplot (ax)
        kind (str): 'speed' or 'stringency'; used to set appropriate labels, 
            colors, and y-axis label
        trim (Bool): Boolean value that indicates whether data should be 
            trimmed; default is False; if True, trims data from the measure
            column (removes values with a z-score >= 3)
    
    Returns:
        Plot
    '''
    if trim == True:
        z = np.abs(stats.zscore(df[df[measure].notnull()][measure]))
        df = df[df[measure].notnull()][(z < 3)]
    if kind == 'speed':
        line_col = '#9F2958'
        line_label = "Regression line (speed)"
        ci_label = '95% confidence interval (speed)'
        y_axis = 'Days from first case'
    else:
        line_col = "lightseagreen"
        line_label = "Regression line (stringency)"
        ci_label = '95% confidence interval (stringency)'
        y_axis = 'Days'
    sns.regplot(x='tourism_to_gdp (%)',
                y=measure,
                data=df,
                ax=ax,
                line_kws={"color": line_col,
                          "alpha":.5,
                          "linewidth":2, 
                          'label':line_label},
                scatter_kws={'s':5,
                             "color":'darkslategray'})
    ax.collections[1].set_label(ci_label)
    ax.set_ylabel(y_axis)
    ax.set_xlabel('Tourism (% of GDP)')
    ax.set_title(title)
    ax.margins(x=0.03,y=0.03)
    slope, intercept, r_value, pv, se = (
        stats.linregress(df[df[measure].notnull()]['tourism_to_gdp (%)'],
                         df[df[measure].notnull()][measure]))
    ax.text(0.95, 0.95, 
            f'Slope: {slope:.2}\nCorr: {r_value:.2}\nP-value: {pv:.2}',
            fontsize=7,
            verticalalignment='top',
            horizontalalignment='right',
            multialignment='left',
            transform=ax.transAxes,
            bbox=dict(edgecolor='gray',
                      facecolor='white',
                      alpha=0.5,
                      linewidth=.5))

def tuple_legend(axes):
    '''
    Set a legend where duplicated labels are shown with handles in a single
    line.
    
    Args:
        axes (tuple or list of Matplotlib Axes instances): the axes to get the 
            legend's labels and handles from
    
    Returns:
        Legend plotted on fig
    '''
    handles = []
    labels = []
    for ax in axes:
        axHandle, axLabel = ax.get_legend_handles_labels()
        handles.extend(axHandle)
        labels.extend(axLabel)
    labels = [fill(l, 20) for l in labels]
    label_dict = defaultdict(list)
    for label, handle in zip(labels, handles):
        label_dict[label].append(handle)
    combined_labels = label_dict.keys()
    tup_handles = [tuple(x) for x in label_dict.values()]
    fig.legend(tup_handles, combined_labels,
               handler_map={tuple: HandlerTuple(ndivide=None)},
               bbox_to_anchor=(0.66, 0.2, 0.5, 0.5))

#Optional/extra function to check interpolation
def check_interpolation():
    '''
    Plots two heatmaps showing travel restrictions by country over time. 
    First heatmap shows original data with a light blue-green color to 
    showcase gaps; second heatmap shows interpolated data.
    
    Args:
        None
    
    Returns:
        Plot
    '''
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,20))
    cbar_ax = fig.add_axes([.91, .3, .01, .4])
    
    def travel_restriction_heatmap(ax, input_df, title):
        variables = ['date', 'c8_international_travel_controls', 'country']

        mask = input_df['country'].isin(
            list(non_null_first_case_gdp['country'].unique()))
        df = input_df[mask][variables]
        df['date'] = df['date'].dt.strftime('%Y-%m-%d')
        tdf = (pd.DataFrame(df[variables].groupby(['date','country'])
                            ['c8_international_travel_controls'].mean())
               .reset_index())
        tdf_piv = tdf.pivot('country',
                            'date',
                            'c8_international_travel_controls')
        sns.heatmap(tdf_piv,
                    vmin=0,
                    vmax=4,
                    ax=ax,
                    cmap='rocket_r',
                    cbar_ax=cbar_ax,
                    yticklabels=True)
        ax.set_title(title)
        ax.yaxis.set_tick_params(labelsize=7)
        ax.set_facecolor('lightseagreen') #makes gaps more obvious
    
    #Pre-interpolation plot
    travel_restriction_heatmap(ax1,
                               df_control_epidem_merged,
                               'Pre-interpolation')
    
    #Post-interpolation plot
    travel_restriction_heatmap(ax2,
                               df_control_epidem_interpolated,
                               'Post-interpolation')
    ax2.axes.yaxis.set_visible(False)
    
    plt.show()

## Import and clean data

### World Bank data (tourism, GDP)

In [3]:
#Load international tourism revenues (USD) from most recent year (2018)
sql_command = """SELECT country, countrycode, value 
                AS tourism_usd_2018 
                FROM world_bank 
                WHERE (year = 2018) 
                    AND (indicator_code='ST.INT.RCPT.CD')"""
df_tourism_usd = pd.read_sql(sql_command, conn)

#Load GDP (USD) from most recent year (2019)
sql_command = """SELECT countrycode, value AS gdp_2019 
                FROM world_bank 
                WHERE (year = 2019) 
                    AND (indicator_code='NY.GDP.MKTP.CD')"""
df_country_gdp = pd.read_sql(sql_command, conn)

### Government measures data

In [4]:
#Import government measures data
sql_command = """SELECT date,
                        country,
                        countrycode,
                        c8_international_travel_controls 
                FROM government_response"""
df_travel_control = pd.read_sql(sql_command, conn)
df_travel_control['date'] = pd.to_datetime(df_travel_control['date'])

### Epidemiology data (cases)

In [5]:
#Load epidemiology data (cases)
sql_command = """SELECT countrycode, date, confirmed 
                FROM epidemiology 
                WHERE source='WRD_WHO'"""
df_epidem = pd.read_sql(sql_command, conn)
df_epidem['date'] = pd.to_datetime(df_epidem['date'])

### Merge and clean dataframes

In [6]:
#Merge travel control and epidemiology data and fill missing travel controls 
#by country; use forward fill to fill travel controls on the assumption that 
#the table is more likely to be updated when the control changes
df_control_epidem_merged = df_travel_control.merge(df_epidem,
                                                   on=['countrycode',
                                                       'date'],
                                                   how='outer')
df_control_epidem_merged.sort_values(by='date', inplace=True)
grp = df_control_epidem_merged.groupby('countrycode')
df_control_epidem_interpolated = grp.apply(
    lambda group: group.interpolate(method='ffill'))
mask = df_control_epidem_interpolated['date'] <= dt.datetime(2021, 10, 1)
df_control_epidem_interpolated = df_control_epidem_interpolated[mask]

#Create new dataframe with one row per country
grp = df_control_epidem_interpolated.groupby('countrycode',
                                             as_index=False)
df_travel_country = grp.agg({'c8_international_travel_controls': 
                             [num_days_any_controls,
                              num_days_2,
                              num_days_3,
                              num_days_4]})
df_travel_country.columns = ["_".join(col_name).rstrip('_')
                             for col_name
                             in df_travel_country.columns.to_flat_index()]

#Remove countries that have no travel control data
df_travel_country['non_null_controls'] = (
    df_travel_country['countrycode'].apply(lambda x: non_null_controls(x)))
mask = df_travel_country['non_null_controls'] == True
null_controls = df_travel_country[~mask]
df_travel_country = df_travel_country[mask]

#Add additional feature columns
s_country = df_travel_country['countrycode']
df_travel_country['first_day_case'] = (
    s_country.apply(lambda x: first_day(x,
                                        'confirmed')))
df_travel_country['first_day_of_travel_controls'] = (
    s_country.apply(lambda x: first_day(x,
                                        'c8_international_travel_controls')))
df_travel_country['first_day_control_2'] = (
    s_country.apply(lambda x: first_day(x,
                                        'c8_international_travel_controls',
                                        greater_than=1)))
df_travel_country['first_day_control_3'] = (
    s_country.apply(lambda x: first_day(x,
                                        'c8_international_travel_controls',
                                        greater_than=2)))
df_travel_country['first_day_control_4'] = (
    s_country.apply(lambda x: first_day(x,
                                        'c8_international_travel_controls',
                                        greater_than=3)))

#Merge with tourism data
df_merged_tourism = (df_tourism_usd.merge(df_country_gdp, on='countrycode')
                     .merge(df_travel_country, on='countrycode'))
df_merged_tourism['tourism_to_gdp (%)'] = (
    df_merged_tourism.apply(lambda row: 
                            row['tourism_usd_2018'] / row['gdp_2019'] * 100,
                            axis=1))

#Add columns with days from first case to travel restriction milestones
s_first_day_case = df_merged_tourism['first_day_case']
s_first_day_control = df_merged_tourism['first_day_of_travel_controls']
s_first_day_control2 = df_merged_tourism['first_day_control_2']
s_first_day_control3 = df_merged_tourism['first_day_control_3']
s_first_day_control4 = df_merged_tourism['first_day_control_4']
df_merged_tourism['days_to_travel_control'] = (s_first_day_control
                                               - s_first_day_case).dt.days
df_merged_tourism['days_to_travel_control2'] = (s_first_day_control2
                                                - s_first_day_case).dt.days
df_merged_tourism['days_to_travel_control3'] = (s_first_day_control3
                                               - s_first_day_case).dt.days
df_merged_tourism['days_to_travel_control4'] = (s_first_day_control4
                                               - s_first_day_case).dt.days

#Sort tourism dataframe by tourism-to-GDP (descending)
df_merged_tourism = df_merged_tourism.sort_values(by=['tourism_to_gdp (%)'],
                                                  ascending=False)

#Filter out any countries with no cases and/or no tourism-to-GDP data
mask = (df_merged_tourism['first_day_case'].notnull()
        & df_merged_tourism['tourism_to_gdp (%)'].notnull())
non_null_first_case_gdp = df_merged_tourism[mask]

In [7]:
#Optional: Call function below to visually check that interpolation 
#makes sense (null values are in light blue-green)

#check_interpolation()

## Descriptive statistics

In [8]:
#Report on null travel control data
valid_country = (df_control_epidem_merged['country']
                 .isin(list(non_null_first_case_gdp['country'].unique())))
valid_date = df_control_epidem_merged['date'] <= dt.datetime(2021, 10, 1)
masked_df = df_control_epidem_merged[(valid_country) & (valid_date)]
null_control_value_count = (
    masked_df['c8_international_travel_controls'].isnull().sum())
non_null_control_value_count = (
    masked_df['c8_international_travel_controls'].notnull().sum())
percent_null = null_control_value_count * 100 / non_null_control_value_count
print(f"Null travel control values: {null_control_value_count}")
print(f"Non-null travel control values: {non_null_control_value_count}")
print(f"Percent null travel control values: {percent_null:.2f}%")

null_controls = (null_controls.merge(df_country_gdp, on='countrycode')
                     .merge(df_tourism_usd, on='countrycode'))
null_controls['tourism_to_gdp (%)'] = (
    null_controls.apply(lambda row:
                        row['tourism_usd_2018'] / row['gdp_2019'] * 100,
                        axis=1))
null_controls_with_tourism = null_controls[null_controls['tourism_to_gdp (%)']
                                           .notnull()]
print("\nCountries without any travel control data:")
display(null_controls_with_tourism[['country', 'tourism_to_gdp (%)']]
        .sort_values(by = 'tourism_to_gdp (%)', ascending=False))

Null travel control values: 863
Non-null travel control values: 88737
Percent null travel control values: 0.97%

Countries without any travel control data:


,country,tourism_to_gdp (%)
1,Antigua and Barbuda,56.199959
7,Maldives,53.305421
6,Saint Lucia,46.597079
4,Grenada,44.619217
5,Saint Kitts and Nevis,34.919371
12,Saint Vincent and the Grenadines,29.198489
13,Samoa,22.488552
9,Montenegro,22.275862
11,São Tomé and Príncipe,16.759258
0,Armenia,9.047158


In [9]:
#Create descriptive table
tourism_desc_df = non_null_first_case_gdp.describe().round(decimals=1)
tourism_desc_df.drop(labels=['tourism_usd_2018', 'gdp_2019'],
                     axis=1,
                     inplace=True)

#Reorder columns so tourism is first
tourism_desc_df = tourism_desc_df[['tourism_to_gdp (%)',
                                   'c8_international_travel_controls_num_days_any_controls', 
                                   'c8_international_travel_controls_num_days_2', 
                                   'c8_international_travel_controls_num_days_3', 
                                   'c8_international_travel_controls_num_days_4', 
                                   'days_to_travel_control', 
                                   'days_to_travel_control2', 
                                   'days_to_travel_control3', 
                                   'days_to_travel_control4']]

#Rename columns to improve readability
mapper = {
    'tourism_to_gdp (%)':
        'Tourism (% of GDP)',
    'c8_international_travel_controls_num_days_any_controls':
        'Number days any travel controls (level 1-4)',
    'c8_international_travel_controls_num_days_2':
        'Number days quarantining or banning arrivals (level 2-4)',
    'c8_international_travel_controls_num_days_3':
        'Number days with travel bans (level 3-4)',
    'c8_international_travel_controls_num_days_4':
        'Number days with total border closure (level 4)',
    'days_to_travel_control':
        'Days from first case to travel controls',
    'days_to_travel_control2':
        'Days from first case to quarantining or banning arrivals (level 2+)',
    'days_to_travel_control3':
        'Days from first case to any travel bans (level 3+)',
    'days_to_travel_control4':
        'Days from first case to total border closure (level 4)'
}
tourism_desc_df.rename(mapper=mapper, axis=1, inplace=True)

#Note: To display table nicely in the PDF version (using LaTeX compilation),
#used code below to convert the DataFrame into a LaTeX table 
#print(tourism_desc_df.to_latex())

## Visualizations

In [10]:
#Visualize travel restrictions on global scale

mask = df_control_epidem_merged['country'].isin(
    list(non_null_first_case_gdp['country'].unique()))
variables = ['date', 'c8_international_travel_controls', 'country']
df = df_control_epidem_merged[mask][variables]
grp = df.groupby(['date', 'c8_international_travel_controls'])
restrictions_by_date = grp.size().unstack(fill_value=0)
perc_restrictions_by_date = (
    restrictions_by_date.divide(restrictions_by_date.sum(axis=1), axis=0) 
    * 100)
col = sns.color_palette("rocket_r")
fig, ax = plt.subplots()
ax.stackplot(perc_restrictions_by_date.index,
             perc_restrictions_by_date[0.0],
             perc_restrictions_by_date[1.0],
             perc_restrictions_by_date[2.0],
             perc_restrictions_by_date[3.0],
             perc_restrictions_by_date[4.0],
             labels=[
                 'No restrictions (level 0)',
                 'Screening arrivals (level 1)',
                 'Quarantine arrivals from \nsome/all regions (level 2)',
                 'Ban arrivals from some \nregions (level 3)',
                 'Ban on all regions or \ntotal border closure (level 4)'
             ],
             colors = col)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
set_mo_yr_ax(ax, minor_rotation=45, labelsize=8)
ax.set_xlim(left=dt.date(2020, 1, 1), right=dt.date(2021, 10, 1))

#Other formatting
plt.legend(loc='center right', bbox_to_anchor=(0.5, 0.5, 1.05, 0.5))
plt.margins(x=0,y=0)
plt.title("Figure 1. Travel restrictions over time", fontsize=12)
plt.ylabel("Share of countries")
plt.savefig("travel-restrictions.pdf", format='pdf', bbox_inches='tight')
plt.close()

In [11]:
#Visualize travel restriction timeline by country

#Turn non-null data into long dataframe for use in visualization
df_tourism_long = pd.melt(non_null_first_case_gdp[['country',
                                                   'days_to_travel_control',
                                                   'days_to_travel_control2',
                                                   'days_to_travel_control3',
                                                   'days_to_travel_control4'
                                                  ]], 
                          id_vars=['country'], 
                          value_vars=['days_to_travel_control',
                                      'days_to_travel_control2',
                                      'days_to_travel_control3',
                                      'days_to_travel_control4'], 
                          var_name='event',
                          value_name='days_since_first_case')

fig, (ax1, ax2) = plt.subplots(1, 2,
                               figsize=(10,20),
                               gridspec_kw={'width_ratios': [3, 1]})

#Create scatterplot to show timeline of travel restrictions by country
sns.scatterplot(y='country', 
                x='days_since_first_case', 
                data=df_tourism_long, 
                ax=ax1, 
                hue='event', 
                palette='rocket_r', 
                s=60, 
                zorder=3)

#Add vertical line for day of first case and horizontal lines for each country
ax1.axvline(0, color='black', lw=.3, zorder=2)
ax1.grid(axis='y', color='gray', lw=.1, zorder=1)

#Horizontal bar chart to show tourism-to-GDP for each country
ax2.barh(non_null_first_case_gdp['country'], 
         non_null_first_case_gdp['tourism_to_gdp (%)'], 
         height=0.8, 
         color='darkslategray')
ax2.invert_yaxis()
ax2.axes.yaxis.set_visible(False)

#Rename legend labels to be more understandable
handles, labels = ax1.get_legend_handles_labels()
label_dict = dict(zip(labels, handles))
new_labels = {'days_to_travel_control':
                  'First day travel controls (level 1)',
              'days_to_travel_control2':
                  'First day quarantining arrivals (level 2)',
              'days_to_travel_control3':
                  'First day any travel bans (level 3)', 
              'days_to_travel_control4':
                  'First day total border closure (level 4)'}
by_label = {new_labels[k]: v for k, v in label_dict.items()}
ax1.legend(by_label.values(), by_label.keys(), loc='upper right')

#Format ticks
ax1.yaxis.set_tick_params(labelsize=7)
ax2.xaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))

#Remove frame for second ax
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['bottom'].set_visible(False)
ax2.spines['left'].set_visible(False)

#Set axis labels
ax1.set_xlabel("Days since first case")
ax2.set_xlabel("Intl tourism to GDP (%)")
ax1.set_ylabel("")

#Set title
suptitle = "Figure 2. Travel restriction timeline vs. international tourism levels"
plt.suptitle(suptitle, y=0.92, fontsize=12)

#Optimize spacing
#Drew on code from https://stackoverflow.com/questions/68498735/how-to-remove-whitespace-on-top-and-bottom-of-seaborn-scatterplots
plt.subplots_adjust(wspace=.02, hspace=0)
miny, nexty, *_, maxy = ax1.get_yticks()
eps = (nexty - miny) / 2
ax1.set_ylim(maxy+eps, miny-eps)
ax2.set_ylim(maxy+eps, miny-eps)

#Move axes & labels to top
ax1.xaxis.set_ticks_position('top')
ax1.xaxis.set_label_position('top')
ax2.xaxis.set_ticks_position('top')
ax2.xaxis.set_label_position('top')

plt.savefig("timeline-vs-tourism.pdf", format='pdf', bbox_inches='tight')
plt.close()

In [12]:
#Scatter plots showing relationship between travel restrictions 
#and tourism level
fig, ax = plt.subplots(2,4, figsize=(12,7))

col = 0
row = 0

stringency_scatter_dict = {
    'days_to_travel_control':
        ['Days until travel controls',
         'speed',
         True],
    'days_to_travel_control2':
        ['Days to quarantining arrivals \n(level 2)',
         'speed',
         True],
    'days_to_travel_control3':
        ['Days to any travel bans \n(level 3)',
         'speed',
         True],
    'days_to_travel_control4':
        ['Days to total border closure \n(level 4)',
         'speed',
         True],
    'c8_international_travel_controls_num_days_any_controls':
        ['Days with controls',
         'stringency',
         False],
    'c8_international_travel_controls_num_days_2':
        ['Days quarantining or banning arrivals \n(level 2-4)',
         'stringency',
         False],
    'c8_international_travel_controls_num_days_3':
        ['Days with travel bans \n(level 3-4)',
         'stringency',
         False],
    'c8_international_travel_controls_num_days_4':
        ['Days with total border closure \n(level 4)',
         'stringency',
         False]
}

for measure, [title, kind, trim] in stringency_scatter_dict.items():
    create_travel_scatter(ax[row, col],
                          non_null_first_case_gdp,
                          measure,
                          title,
                          kind,
                          trim=trim)
    if col == 3:
        row +=1
        col = 0
    else:
        col +=1

tuple_legend((ax[0,0], ax[1,0]))

plt.suptitle("Figure 3. Travel restrictions vs. international tourism (% of GDP)",
             fontsize=12)
plt.tight_layout()
plt.savefig("restrictions_scatter.pdf", format='pdf', bbox_inches='tight')
plt.close()

# Code for Part 2

## Set-up

### Import modules and set settings

In [13]:
#Import necessary packages
import mechanize
import tarfile
from pathlib import Path
import pandas as pd
import json
from os import sep
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk import NaiveBayesClassifier,classify
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.legend_handler import HandlerTuple
import matplotlib.dates as mdates
import psycopg2
import re
import reverse_geocoder
import datetime as dt
from collections import Counter,defaultdict
from textwrap import fill

#Set directory path
datadir = Path().cwd()

#Set up connection to COVID database
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19',
    user='covid19',
    password='covid19')
cur = conn.cursor()

#Don't print SettingWithCopyWarning
pd.options.mode.chained_assignment = None

### Functions

In [14]:
#Note: The json_to_df function draws on code from 
#https://towardsdatascience.com/converting-yelp-dataset-to-csv-using-pandas-2a4c8f03bd88
def json_to_df(filename, dtypes, columns, date_query=False):
    '''
    Opens JSON files in chunks and drops unnecessary data during import process.
    
    Args:
        filename (str): name of JSON file to read
        dtypes (dict or Bool): argument for dtype parameter when calling
            Pandas read_json function
        columns (list of str): list of columns to return in output DataFrame
        date_query (Bool): Boolean value that queries 'date' column for values
            on or after 1 Jan 2020 if True; default value is False
    
    Returns:
        Pandas DataFrame
    '''
    temp_df = []
    with open(datadir / filename, "r") as f:
        reader = pd.read_json(f, orient="records", lines=True,
                              dtype=dtypes, chunksize=1000)
        
        for chunk in reader:
            if date_query == True:
                reduced_chunk = chunk[columns].query("`date` >= '2020-01-01'")
            else:
                reduced_chunk = chunk[columns]
            temp_df.append(reduced_chunk)
    
    temp_df = pd.concat(temp_df, ignore_index=True)
    return temp_df

def safe_regex(keyword_list):
    '''
    Escapes any special characters for all keywords in a list
    
    Args:
        keyword_list (list of str): list of keywords
    
    Returns:
        list of str
    '''
    safe_list = [re.escape(item) for item in keyword_list]
    return safe_list

#The following function draws on code from Ch. 11 of Bernie Hogan's Fundamentals 
#of Social Data Science textbook (draft version)
eng_stopwords = set(stopwords.words('english'))
def clean_text(text):
    '''
    Cleans text and returns tokens in a single string
    
    Args:
        text (str): string to clean
    
    Returns:
        string
    
    '''
    text = text.lower()
    tokens = [x for x in wordpunct_tokenize(text) if x.isalpha()]
    tokens = [word for word in tokens if word not in eng_stopwords]
    tokens = [WordNetLemmatizer().lemmatize(word,pos='v') for word in tokens]
    return " ".join(tokens)

cov_crisis_words = ['covid', 'coronavirus', 'pandemic', 'corona', 'virus']
cov_measures_words = ['mask', 'distance', 'sanitize', 'gloves', 'sanitizer',
                      'quarantine', 'enforce', 'measure']
cov_cancel_words = ['refund', 'cancel', 'cancellation']
cov_service_change_words = ['curbside', 'deliver', 'contactless', 'delivery']
def create_word_dict(x):
    '''
    Identifies how many COVID-related words of 4 different categories are in a 
    string. Categories are: words relating to the COVID crisis itself (e.g. 
    COVID, pandemic), words relating to measures taken to reduce the spread of 
    COVID (e.g. mask, sanitize), words relating to cancellations, and words 
    related to services that became much more common during the pandemic (e.g. 
    curbside, contactless). Full word lists are in the variables 
    cov_crisis_words, cov_measures_words, cov_cancels_words, and 
    cov_service_change_words
    
    Args:
        x (str): string to check
    
    Returns:
        dictionary with counts of different categories of COVID-related words 
    '''
    cov_dict = {'cov_crisis_words': 0, 'cov_measures_words': 0, 
                'cov_cancel_words': 0, 'cov_service_change_words': 0}
    for i in x.split():
        if i in cov_crisis_words: 
            cov_dict['cov_crisis_words'] += 1
        elif i in cov_measures_words: 
            cov_dict['cov_measures_words'] += 1
        elif i in cov_cancel_words: 
            cov_dict['cov_cancel_words'] += 1
        elif i in cov_service_change_words: 
            cov_dict['cov_service_change_words'] += 1
    return cov_dict

def location_to_countrycode(postal_code, coordinates):
    '''
    Takes postal code and matches it to either Canada or US. If postal code is 
    missing, takes tuple of latitude and longitude and returns country code 
    using reverse_geocoder package (uses the reverse_geocoder package rather 
    than the Google API due to limits in the number of queries Google allows 
    per day).
    
    Args:
        postal_code (str): string corresponding to a US or Canadian postal code
        coordinates (tup): tuple containing latitude and longitude
    
    Returns:
        string (3-letter country code)
    '''
    stripped_postal = postal_code.replace(" ", "")
    if len(stripped_postal) == 5:
        return 'USA'
    elif len(stripped_postal) == 6 or len(stripped_postal) == 3:
        return 'CAN'
    else:
        countrycode = reverse_geocoder.search(coordinates)[0]['cc']
        if countrycode == 'US':
            return 'USA'
        elif countrycode == 'CA':
            return 'CAN'
        else:
            return countrycode
    
def set_mo_yr_ax(ax, minor_rotation=0, labelsize=12):
    '''
    Formats x-axis with years as major ticks and months as minor ticks. Offers 
    option to rotate the months and change the size of the month/year labels.
    
    Args:
        ax (Matplotlib Axes instance): ax to set x-axis for
        minor_rotation (float): number between 0 and 360; sets month rotation;
            default value is 0 (no rotation)
        labelsize (float): fontsize of month and year labels
    
    Returns:
        Formatted x-axis
    '''
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    for tick in ax.xaxis.get_major_ticks():
        tick.tick1line.set_markersize(0)
        tick.tick2line.set_markersize(0)
        tick.label1.set_horizontalalignment('center')
        tick.set_pad(4.5 * tick.get_pad())
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.remove_overlapping_locs = False
    ax.tick_params(axis="x", which="minor", rotation=minor_rotation,
                   labelsize=labelsize)
    ax.tick_params(axis="x", which="major", labelsize=labelsize)
    
def plot_review_count(time_df, ax, category=None):
    '''
    Creates stackplot showing review count over time
    
    Args:
        time_df (DataFrame): Must be a DataFrame with a DateTime index
        ax (Matplotlib Axes instance): ax to plot on
        category (str): column corresponding to a tourism business category
            to filter on; optional (default value is None)
    
    Returns:
        Plot
    '''
    if category != None:
        resampled = time_df[time_df[category] == True].resample('W')
        covid_review_count = (resampled['covid_words_bool'].sum())
        non_covid_review_count = (resampled['covid_words_bool'].count()
                                  - resampled['covid_words_bool'].sum())
    else:
        resampled = time_df.resample('W')
        covid_review_count = resampled['covid_words_bool'].sum()
        non_covid_review_count = (resampled['covid_words_bool'].count()
                                  - resampled['covid_words_bool'].sum())
    ax.stackplot(covid_review_count.index,
                 covid_review_count,
                 non_covid_review_count,
                 labels = ["Includes COVID-related terms",
                           "Doesn't include COVID-related terms"], 
                 colors = ['#9F2958', 'lightseagreen'])
    ax.yaxis.set_major_formatter(mtick.EngFormatter())
    
def plot_avg_rating(time_df, ax, resample='W', category=None):
    '''
    Creates lineplot showing average rating over time. Includes options to 
    change resampling period and choose a specific category to filter the 
    time_df on.
    
    Args:
        time_df (DataFrame): Must be a DataFrame with a DateTime index
        ax (Matplotlib Axes instance): ax to plot on
        resample (str): string corresponding to DateTime resampling parameter;
            default value is 'W' (weekly resampling)
        category (str): column corresponding to a tourism business category
            to filter on; optional (default value is None)
    
    Returns:
        Plot
    '''
    if category != None:
        time_df = time_df[time_df[category] == True]
    cov_words_true = time_df[time_df['covid_words_bool'] == True]
    cov_words_false = time_df[time_df['covid_words_bool'] == False]
    ax.plot(cov_words_true.resample(resample)['stars'].mean(),
            color='#9F2958',
            label='Includes COVID-related terms')
    ax.plot(cov_words_false.resample(resample)['stars'].mean(),
            color='lightseagreen',
            label="Doesn't include COVID-related terms")
    ax.plot(time_df.resample(resample)['stars'].mean(),
            color='gray',
            label="All reviews")
    ax.set_ylim(1,5)
    ax.set_yticks([1,2,3,4,5])

def covid_case_plot(ax, xlim_ref=ax):
    '''
    Creates a plot of US daily case counts over time. xlim_ref can be used
    to ensure subplots within a single chart have a consistent x-axis
    
    Args:
        ax (Matplotlib Axes instance): ax to plot on
        xlim_ref (Matplotlib Axes instance): default value is the Axes instance
            being plotted; specifies the ax to take the xlim from
    
    Returns:
        Plot
    '''
    df_us_cases_resampled = df_us_cases.resample('W')['daily_confirmed'].mean()
    ax.bar(df_us_cases.index,
           df_us_cases['daily_confirmed'],
           color='gainsboro',
           label='Daily reported cases')
    ax.plot(df_us_cases_resampled,
            color='gray',
            lw=.5,
            label='7-day average')
    
    #Format graph
    ax.set_ylabel("Daily cases")
    ax.set_yticks([150000, 300000])
    ax.yaxis.set_major_formatter(mtick.EngFormatter())
    ax.set_xlim(xlim_ref.get_xlim())
    ax.margins(x=0.02,y=0)
    
    #Add annotation for 7-day average
    annotation_date = dt.datetime(2020, 9, 13)
    point = df_us_cases_resampled.loc[annotation_date]
    ax.annotate('7-day\navg.',
                (annotation_date, point),
                xytext=(annotation_date, 100000),
                ha="center",
                arrowprops=dict(arrowstyle="-", color='gray', lw=.5))

    #Remove frame
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.axes.xaxis.set_visible(False)

def tuple_legend(axes):
    '''
    Set a legend where duplicated labels are shown with handles in a single
    line.
    
    Args:
        axes (tuple or list of Matplotlib Axes instances): the axes to get the 
            legend's labels and handles from
    
    Returns:
        Legend plotted on fig
    '''
    handles = []
    labels = []
    for ax in axes:
        axHandle, axLabel = ax.get_legend_handles_labels()
        handles.extend(axHandle)
        labels.extend(axLabel)
    labels = [fill(l, 20) for l in labels]
    label_dict = defaultdict(list)
    for label, handle in zip(labels, handles):
        label_dict[label].append(handle)
    combined_labels = label_dict.keys()
    tup_handles = [tuple(x) for x in label_dict.values()]
    fig.legend(tup_handles, combined_labels,
               handler_map={tuple: HandlerTuple(ndivide=None)},
               bbox_to_anchor=(0.66, 0.2, 0.5, 0.5))

def yelp_review_heatmap(ax, df, title, set_ylim=None, cbar_ax=None,
                        count_col='covid_words_count'):
    '''
    Creates a heatmap of the average rating over time by word count. set_ylim 
    and cbar_ax can be used to ensure subplots within a single chart have a 
    consistent y-axis and colorbar, respectively. count_col can be used to set 
    which column used on the y-axis (word count).
    
    Args:
        ax (Matplotlib Axes instance): ax to plot on
        df (DataFrame): DataFrame with data to plot
        title (str): title for subplot
        set_ylim (tup): tuple with min and max for y-axis; optional (default
            value is None, which means ylim is set automatically)
        cbar_ax (Matplotlib Axes instance): default value is None, meaning 
            colorbar takes space from the main Axes; if not None, specifies
            the ax in which to draw the colorbar
    
    Returns:
        Plot
    '''
    
    #Set up x-axis labels (convert from week-year to month)
    start_date = dt.date(2020, 1, 1)
    end_date = dt.date(2021, 1, 28)
    m = 1
    y = 2020
    weeks_w_first_day_of_mo = {}
    labels = []
    while start_date <= end_date:
        if start_date.month == 1:
            weeks_w_first_day_of_mo[start_date.strftime('%Y01')] = (
                start_date.strftime('%b\n%Y'))
        else:
            weeks_w_first_day_of_mo[start_date.strftime('%Y%V')] = (
                start_date.strftime('%b'))
        if m == 12:
            m = 1
            y += 1
        else:
            m +=1
        start_date = dt.date(y, m, 1)
    for week_year in sorted(list(df['week_year'].unique())):
        if week_year in weeks_w_first_day_of_mo.keys():
            labels.append(weeks_w_first_day_of_mo[week_year])
        else:
            labels.append('')
    
    #Create heatmap
    variables = ['week_year', 'stars', count_col]
    tdf = (pd.DataFrame(df[variables].groupby(['week_year',count_col])
                        .filter(lambda x: len(x) > 5)
                        .groupby(['week_year',count_col])['stars'].mean())
           .reset_index())
    tdf_pivot = tdf.pivot(count_col, 'week_year', 'stars')
    
    sns.heatmap(tdf_pivot,
                vmin=1,
                vmax=5,
                xticklabels=labels,
                cbar_kws={'label': 'Average rating', 'ticks':[1, 2, 3, 4, 5]},
                ax=ax,
                cbar_ax=cbar_ax)
    ax.set_title(title)
    ax.invert_yaxis()
    if set_ylim != None:
        ax.set_ylim(set_ylim[0],set_ylim[1])
    ax.tick_params(axis="both", rotation=0, bottom=False, left=False)
    ax.set(xlabel=None, ylabel="Count of COVID-related words")

## Import and clean data

### Epidemiology data (cases)

In [15]:
#Load epidemiology data (cases & deaths)
sql_command = """SELECT countrycode, date, confirmed 
                FROM epidemiology 
                WHERE source='WRD_WHO'
                AND countrycode='USA'"""
df_us_cases = pd.read_sql(sql_command, conn)
df_us_cases['date'] = pd.to_datetime(df_us_cases['date'])
df_us_cases = df_us_cases.sort_values(by=['date'], ascending=True)
df_us_cases = df_us_cases.set_index('date')
df_us_cases['daily_confirmed'] = df_us_cases['confirmed'].diff()

### Date of workplace closures

In [16]:
#Import government measures data around workplace closing
#Level 2 closure means requiring closing for some sectors/categories of workers
sql_command = """SELECT date, c2_workplace_closing
                FROM government_response WHERE countrycode='USA'"""
df_us_control = pd.read_sql(sql_command, conn)
df_us_control['date'] = pd.to_datetime(df_us_control['date'])

mask = df_us_control['c2_workplace_closing'] > 1
required_closure = df_us_control[mask]['date'].min()

### Download Yelp data from web

In [17]:
#Download JSON files from Yelp
#Full dataset documentation here:
#https://www.yelp.com/dataset/documentation/main
#WARNING: This code downloads a zipped folder that is 4.9GB compressed 
#and 10.9GB uncompressed.

br = mechanize.Browser()

#Avoid being blocked as a robot
br.set_handle_robots(False)
br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36')]

#Fill out the user consent form required by Yelp to download the dataset
#Note that name, email, and signature have been masked for anonymity here
br.open("https://www.yelp.com/dataset/download")
br.select_form(nr=0)
br.form['name'] = 'NAME'
br.form['email'] = 'EMAIL'
br.form['signature'] = 'INITIALS'
br.form['terms_accepted'] = ['y',]
br.submit()

#Submitting the form takes the user to a page with options to download either 
#JSON data, COVID-19-specific data, or photos (with download links only valid 
#for 30 seconds). This code downloads a .tgz zip file, which has JSON files 
#with data on businesses, reviews, checkins, tips, and photos.
tgz_file = 'yelp_dataset.tgz'
link = br.find_link(text='Download JSON')
br.retrieve(link.url, tgz_file)[0]

#Unpack the .tgz file so that the JSON files can be accessed
tar = tarfile.open(tgz_file, "r:gz")
tar.extractall()
tar.close()

#Set the right directory
datadir = Path().cwd()

### Import Yelp business and review data

In [18]:
business_filename = 'yelp_academic_dataset_business.json'
reviews_filename = 'yelp_academic_dataset_review.json'

#Specify reviews dtypes here in order to minimize time to import the very large 
#reviews file. Business file is smaller so dtypes can be inferred without 
#drastically increasing the import time 
business_dtypes = True
reviews_dtypes = {"stars": np.int16, 
                  "useful": np.int32,
                  "funny": np.int32,
                  "cool": np.int32, 
                  "date":str, 
                  "text":str, 
                  "review_id":str, 
                  "user_id": str, 
                  "business_id": str}

#Specify which columns to import
business_columns = ['business_id', 'city', 'state', 'postal_code',
                    'latitude', 'longitude', 'categories']
reviews_columns = ['review_id', 'business_id', 'stars', 'text', 'date']

business_df = json_to_df(business_filename, business_dtypes, business_columns)
reviews_df = json_to_df(reviews_filename, reviews_dtypes, reviews_columns,
                        date_query=True)

### Clean and merge dataframes

In [19]:
#Clean and merge business and reviews dataframes

reviews_df['date'] = pd.to_datetime(reviews_df['date'])

#Identify which businesses are tourism-related
hotel_keywords = ['Hotel', 'Resorts', 'Vacation Rentals', 'Bed & Breakfast']
activity_keywords = ['Tours', 'Museums', 'Festivals', 'Landmarks', 
                     'Arts & Entertainment', 'Amusement Parks', 'Zoos']
transport_keywords = ['Train', 'Transportion', 'Airlines', 'Airport', 'Taxis']
dining_keywords = ['Food', 'Restaurants', 'Bars', 'Pubs']
tourism_category_dict = {'hotel' : hotel_keywords,
                         'activity' : activity_keywords,
                         'transport' : transport_keywords,
                         'dining' : dining_keywords}

for category, keywords in tourism_category_dict.items():
    business_df[category] = (
        business_df['categories'].str.contains('|'.join(safe_regex(keywords))))

tourism_cat_list = ['hotel', 'activity', 'transport', 'dining']
tourism_business_df = business_df[business_df[tourism_cat_list].any(axis=1)]

#Identify each business' country
tourism_business_df['countrycode'] = (
    tourism_business_df.apply(lambda row: 
                              location_to_countrycode(row['postal_code'],
                                                      (row['latitude'],
                                                       row['longitude'])),
                              axis=1))

#Inner merge business details with the review DataFrame
merged_reviews_df = reviews_df.merge(tourism_business_df, on='business_id')
merged_reviews_df.rename(columns={"date": "review_date"}, inplace=True)

#Reduce to US reviews only
mask = merged_reviews_df['countrycode'] == 'USA'
merged_reviews_df_us = merged_reviews_df[mask]

#Clean text after merging & selecting only US reviews to avoid cleaning text 
#on unnecessary reviews (i.e. for Canada and/or non-tourism businesses)
merged_reviews_df_us['clean_text'] = (
    merged_reviews_df_us["text"].map(lambda x: clean_text(x)))

#Add column to note reviews before/after COVID (using March 1 as the separator)
merged_reviews_df_us['before_cov'] = (
    merged_reviews_df_us['review_date']
    .apply(lambda x: True if (x < dt.datetime(2020, 3, 1)) else False))

Loading formatted geocoded file...


In [20]:
#Naive Bayes: See which words can be used to predict whether a review was 
#before or after COVID. Use to ensure that I'm not missing any key words 
#associated with COVID
#Draws on code from Chapter 11 (NLP) of Bernie Hogan's lecture notes

#Select sample of data (50K reviews) and create a list of tokens
featureset_data = (
    merged_reviews_df_us.sample(n=50000, random_state = 42)[['clean_text',
                                                             'before_cov']])
featureset_data['clean_tokens'] = (
    featureset_data['clean_text'].apply(lambda x: x.split(" ")))

#Select a smaller set of words to run model against
all_words =  pd.Series(Counter(featureset_data[:10000]["clean_tokens"].sum()))
top15_words = frozenset(all_words[all_words > 14].index)

features = featureset_data["clean_tokens"].map(lambda x: 
                {i:i in set(x) for i in top15_words})

featurelist = list(zip(features, featureset_data["before_cov"]))

#Split into train and test set (50-50 split)
train_set, test_set = featurelist[:25000], featurelist[25000:]
classifier = NaiveBayesClassifier.train(train_set)

print(f"Number of words used in model: {len(all_words[all_words > 14])}")
print(featureset_data["before_cov"].value_counts())
print(classify.accuracy(classifier, test_set))
print(classifier.show_most_informative_features(20))

Number of words used in model: 2812
False    37587
True     12413
Name: before_cov, dtype: int64
0.71996
Most Informative Features
                pandemic = True            False : True   =    138.1 : 1.0
                    mask = True            False : True   =    116.1 : 1.0
              quarantine = True            False : True   =     26.5 : 1.0
                 enforce = True            False : True   =     18.7 : 1.0
                curbside = True            False : True   =     18.2 : 1.0
                   virus = True            False : True   =     17.6 : 1.0
             coronavirus = True            False : True   =     16.7 : 1.0
                sanitize = True            False : True   =     15.2 : 1.0
               valentine = True             True : False  =     13.0 : 1.0
              guidelines = True            False : True   =     11.5 : 1.0
               sanitizer = True            False : True   =     11.5 : 1.0
                distance = True            F

In [21]:
#Count how many COVID-related words in each post, by type of word
merged_reviews_df_us['cov_word_dict'] = (
    merged_reviews_df_us['clean_text'].apply(create_word_dict))
merged_reviews_df_us.reset_index(inplace=True)
cov_word_df = pd.json_normalize(merged_reviews_df_us['cov_word_dict'])
merged_reviews_df_us = merged_reviews_df_us.join(cov_word_df)

#Determine how many COVID-related words were used in each post 
merged_reviews_df_us['covid_words_count'] = (
    merged_reviews_df_us[['cov_crisis_words',
                          'cov_measures_words',
                          'cov_cancel_words',
                          'cov_service_change_words']]
    .sum(axis=1))

#Determine if COVID-related words were used in each post 
merged_reviews_df_us['covid_words_bool'] = (
    merged_reviews_df_us['covid_words_count'].apply(lambda x: x > 0))

#Add column with just the week and year for heatmap grouping later
merged_reviews_df_us['week_year'] = (
    merged_reviews_df_us['review_date'].dt.strftime('%Y%V'))

#Create time-indexed DataFrame for visualizations
time_df_us = merged_reviews_df_us[['covid_words_bool',
                                   'covid_words_count',
                                   'stars',
                                   'review_date',
                                   'hotel',
                                   'activity',
                                   'transport',
                                   'dining',
                                   'cov_crisis_words',
                                   'cov_measures_words',
                                   'cov_cancel_words',
                                   'cov_service_change_words']]
time_df_us.set_index('review_date',inplace=True)

## Descriptive statistics

In [22]:
last_review_date = merged_reviews_df_us['review_date'].max()

print(f"Number businesses in original Yelp download: {len(business_df)}")
print(f"Number unique business categories: {business_df['categories'].nunique()}")
print(f"Number tourism businesses: {len(tourism_business_df)}")
print(f"Number reviews in original Yelp download: {len(reviews_df)}")
print(f"Number reviews of tourism businesses: {len(merged_reviews_df)}")
print(f"Number reviews of US tourism businesses: {len(merged_reviews_df_us)}")
print(f"Final available review date: {last_review_date.strftime('%B %d, %Y')}")
print(f"First day workplace closure: {required_closure.strftime('%B %d, %Y')}")

Number businesses in original Yelp download: 160585
Number unique business categories: 88115
Number tourism businesses: 77985
Number reviews in original Yelp download: 646352
Number reviews of tourism businesses: 478271
Number reviews of US tourism businesses: 450547
Final available review date: January 28, 2021
First day workplace closure: March 16, 2020


## Visualizations

In [23]:
#Show review count and rating over time vs. case count
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2,
                                             figsize=(12, 6),
                                             gridspec_kw={'height_ratios': 
                                                          [1, 3]})
    
#Plot review counts over time
plot_review_count(time_df_us, ax3)
ax3.set_ylabel("Weekly review count")
ax3.margins(x=0.02,y=0)
set_mo_yr_ax(ax3)
ax3.axvline(required_closure, color='black', linestyle=':', lw=1)

#Plot average rating
plot_avg_rating(time_df_us, ax4)
ax4.margins(x=0.02,y=0)
ax4.axvline(required_closure,
            color='black',
            linestyle=':',
            lw=1,
            label='Workplace closures')
ax4.set_ylabel("Stars")
set_mo_yr_ax(ax4)

#Plot case counts
for ax in [ax1, ax2]:
    covid_case_plot(ax, xlim_ref=ax3)

#Set legend and titles
tuple_legend((ax3, ax4))
ax1.set_title("Review count vs. cases over time")
ax2.set_title("Review ratings vs. cases over time")
plt.suptitle("Figure 4. Yelp reviews of tourism businesses in the US over time",
             y=1.03)

#Adjust spacing
fig.tight_layout(pad=2.0)
plt.subplots_adjust(hspace=0.01)

plt.savefig("yelp_reviews_all.pdf", format='pdf', bbox_inches='tight')
plt.close()

In [24]:
#Plot count & ratings by COVID word category
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2,
                                             figsize=(12, 6),
                                             gridspec_kw={'height_ratios': 
                                                          [1, 3]})
        
category_dict = {'cov_crisis_words':
                     "Reviews with words around COVID crisis", 
                 'cov_measures_words':
                     "Reviews with words around COVID measures", 
                 'cov_cancel_words':
                     "Reviews with words around cancellations/ refunds", 
                 'cov_service_change_words':
                     "Reviews with words around service changes"}

#Plot % of reviews that mention COVID-related words by category
noncovid_review_counts = time_df_us.resample('W')['covid_words_bool'].count()
for category, title in category_dict.items():
    resampled = time_df_us[time_df_us[category] > 0].resample('W')
    covid_review_count = resampled['covid_words_bool'].sum()
    percent = covid_review_count / noncovid_review_counts * 100
    ax3.plot(percent, label=title)

#Plot average rating of reviews that mention COVID-related words by category
for category, title in category_dict.items():
    resampled = time_df_us[time_df_us[category] > 0].resample('W')
    ax4.plot(resampled['stars'].mean(), label=title)

#Add control lines
for ax in [ax3, ax4]:
    ax.axvline(required_closure, color='black', linestyle=':', lw=1,
               label='Workplace closures')

#Subplot formatting
set_mo_yr_ax(ax3)
ax3.set_ylabel("% of total reviews (weekly)")
ax3.margins(x=0.02,y=0)
ax3.set_ylim(0,18)
ax3.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax4.set_ylim(1,5)
ax4.set_yticks([1,2,3,4,5])    
set_mo_yr_ax(ax4)
ax4.set_xlim(ax3.get_xlim())
ax4.set_ylabel("Stars")
ax4.margins(x=0.02,y=0)

#Plot case counts
for ax in [ax1, ax2]:
    covid_case_plot(ax, xlim_ref=ax3)

#Set legend and titles
lines, labels = ax3.get_legend_handles_labels()
labels = [fill(l, 20) for l in labels]
fig.legend(lines, labels, bbox_to_anchor=(0.65, 0.2, 0.5, 0.5))
ax1.set_title("Mentions of different COVID-related words vs. cases over time")
ax2.set_title("Review ratings vs. cases over time")
plt.suptitle("Figure 5. Yelp tourism reviews that mention COVID-related words",
             y=1.03)

#Set spacing
fig.tight_layout(pad=2.0)
plt.subplots_adjust(hspace=0.01)

plt.savefig("yelp_reviews_covid.pdf", format='pdf', bbox_inches='tight')
plt.close()

In [25]:
#Heatmap showing average review by number of COVID words and review date 
#(grouped by week). Filtered out week/word count combinations with fewer than 
#5 reviews in order to avoid showing non-representative groupings
fig, ax = plt.subplots(figsize=(12,4))
yelp_review_heatmap(ax,
                    merged_reviews_df_us,
                    "Figure 6. Average rating of tourism reviews by number of COVID-related words")
plt.savefig("heatmap_all.pdf", format='pdf', bbox_inches='tight')
plt.close()

In [26]:
#Heatmap with average star rating by COVID words per week by COVID word category
#Filtered out week/word count combinations with fewer than 5 reviews in order 
#to avoid showing non-representative groupings

fig, ax = plt.subplots(2,2, figsize=(12,7))
col_indexer = 0
row_indexer = 0
cbar_ax = fig.add_axes([.91, .3, .01, .4])

category_dict = {'cov_crisis_words': "Words around COVID crisis", 
                 'cov_measures_words': "Words around COVID measures", 
                 'cov_cancel_words': "Words around cancellations/refunds", 
                 'cov_service_change_words': "Words around service changes"}

for category, title in category_dict.items():
    yelp_review_heatmap(ax[col_indexer, row_indexer], 
                        merged_reviews_df_us, 
                        f"{title}", 
                        cbar_ax=cbar_ax, 
                        count_col=category, 
                        set_ylim=(0,8))
    if col_indexer == 1:
        col_indexer = 0
        row_indexer += 1
    else:
        col_indexer += 1
    
plt.subplots_adjust(hspace=.5)
plt.suptitle("Figure 7. Average rating by date and COVID word count (weekly) by COVID word category")
plt.savefig("heatmap_cov_category.pdf", format='pdf', bbox_inches='tight')
plt.close()

In [27]:
#Plot count & ratings by type of tourism business
category_list = ['dining', 'hotel', 'activity', 'transport']

fig, ax = plt.subplots(2,4, figsize=(12,5))

col_indexer = 0
for category in category_list:
    plot_review_count(time_df_us, ax[0, col_indexer], category=category)
    ax[0, col_indexer].set_title(f"{category.capitalize()} reviews")
    set_mo_yr_ax(ax[0, col_indexer], minor_rotation=45, labelsize=7)
    
    #Add line for required workplace closures (with label)
    ax[0, col_indexer].axvline(required_closure,
                               color='black',
                               linestyle=':',
                               lw=1,
                               label='Workplace closures')
    
    col_indexer += 1

col_indexer=0
for category in category_list:
    plot_avg_rating(time_df_us,
                    ax[1, col_indexer],
                    resample='M',
                    category=category)
    set_mo_yr_ax(ax[1, col_indexer], minor_rotation=45, labelsize=7)
    ax[1, col_indexer].set_xlim(ax[0,0].get_xlim())
    
    #Add line for required workplace closures (no label)
    ax[1, col_indexer].axvline(required_closure,
                               color='black',
                               linestyle=':',
                               lw=1)
    
    col_indexer += 1

fig.tight_layout(pad=0.9)

#Set labels and legend
ax[0,0].set_ylabel("Weekly review count")
ax[1,0].set_ylabel("Stars")
plt.suptitle("Figure 8. Count and average rating by type of tourism business over time",
             y=1.03)
tuple_legend((fig.axes[0], fig.axes[-1]))

plt.savefig("tourism_category.pdf", format='pdf', bbox_inches='tight')
plt.close()

In [28]:
#Heatmap with average star rating by COVID words per week by category. Filtered 
#out week/word count combinations with fewer than 5 reviews in order to avoid 
#showing non-representative groupings

fig, ax = plt.subplots(2,2, figsize=(12, 7))
col_indexer = 0
row_indexer = 0
cbar_ax = fig.add_axes([.91, .3, .01, .4])

for category in category_list:
    mask = merged_reviews_df_us[category] == True
    category_masked_df = merged_reviews_df_us[mask]
    yelp_review_heatmap(ax[col_indexer, row_indexer], 
                        category_masked_df, 
                        f"{category.capitalize()} reviews", 
                        set_ylim=(0,10), 
                        cbar_ax=cbar_ax)
    if col_indexer == 1:
        col_indexer = 0
        row_indexer += 1
    else:
        col_indexer += 1
    
plt.subplots_adjust(hspace=.5)
plt.suptitle("Figure 9. Average rating by date and COVID word count (weekly) by category")

plt.savefig("heatmap_tourism_category.pdf", format='pdf', bbox_inches='tight')
plt.close()

# Introduction

Prior to COVID-19, tourism was one of the most important sectors globally, accounting for 10% of global GDP and 334 million jobs (World Travel &  Tourism Council, 2021). The emergence of COVID-19 quickly threw the tourism sector into disarray, with far-reaching and extreme impacts: It put 100-120 million direct tourism jobs at stake (UNWTO, 2021a) and had a negative upstream effect into may sectors due to the lack of demand for intermediate goods and services (UNCTAD, 2021). Global tourist arrival declined by 74% year-on-year, corresponding with an estimated loss of USD 1.3 trillion in export revenues (UNWTO, 2021b).

Tourism companies were impacted both by governmental restrictions—including strict border controls, restrictions on international travel, and mandatory workplace closures (Gössling et al., 2021; Hall et al., 2020; UNWTO, 2020)—as well as by increased customer anxieties (BCG, 2020; McKinsey & Company, 2020). However, tourism businesses were not static entities during this time: As COVID-19 created a new normal, the tourism sector sought at times to lessen restrictions (e.g. Aratani, 2021) and modified the way they did business, from implementing curbside pickup to limiting capacity to enable social distancing (Deloitte, 2020, p. 8-9). Much of the COVID-19-related tourism literature has focused on the implications of travel restrictions and other measures on tourism (Gössling et al., 2021). To supplement these findings, this paper explores facets of a more complex relationship between the tourism sector, consumers, and governments during COVID-19. 

This paper is split into two parts. Part 1 examines whether pre-COVID international tourism revenues (as percent of GDP) impacted the speed or stringency of countries’ international travel restrictions, using data from the OxCOVID19 Database (Mahdi et al., 2021). Part 2 then explores how COVID-19 impacted Yelp reviews of tourism businesses in the United States.

# Part 1: International travel restrictions

## Introduction

Throughout the COVID-19 pandemic, politicians have largely justified travel restrictions as health interventions. However, decisions to implement travel restrictions were also influenced by non-health factors, including domestic politics, foreign policy, and/or geopolitical strategies (Seyfi et al., 2020). In order to better understand how governments attempted to balance economic well-being versus their citizens' health, RQ1 examines the relationship between the size of countries’ tourism sectors and their implementation of international travel controls: 

> $RQ_1$: Did countries’ dependence on international tourism impact how quickly or stringently they imposed international travel restrictions? 

During the pandemic, tourism-dependent countries were hit hard by the sharp decrease in international tourism receipts (Brookings, 2021). Additionally, the travel industry actively attempted to influence government policies. For example, in April 2021, the Washington Post documented how the travel industry had lobbied the US government for months to remove international travel restrictions for visitors from the Schengen area (Aratani, 2021). Therefore, I hypothesize that:

> $H_1$: Countries with a higher dependency on international tourism implemented travel restrictions *more slowly* than countries with a lower dependency on international tourism.  

> $H_2$: Countries with a higher dependency on international tourism implemented *less stringent* travel restrictions than countries with a lower dependency on international tourism.

## Results 

To address RQ1, I use data from the epidemiology, government response, and World Bank tables of the OxCOVID19 Database (Mahdi et al., 2021). To provide a fair comparison of international tourism revenues by country, I load both international tourism revenues and gross domestic product (GDP) in current USD from the most recent pre-COVID year available. Note that as the OxCOVID19 Database does not include full historical data from the World Bank (2018, 2019) and this summative disallows using data from outside the OxCOVID19 Database for Part 1, I needed to normalize 2018 international tourism revenue data using 2019 GDP data. To assess the speed and stringency of travel restrictions, I import the C8 International Travel Controls indicator from the government response table, which is sourced from Oxford COVID-19 Government Response Tracker (Hale et al., 2020). This indicator provides the level of a government’s international travel restrictions in an ordinal scale: 0 for no restrictions, 1 for screening arrivals, 2 for quarantine arrivals from some or all regions, 3 for ban arrivals from some regions, and 4 for ban on all regions or total border closure. Lastly, as COVID-19 reached different countries at different times, with implications on their restriction rollout timeline (Brookings, 2020), I import cumulative daily confirmed cases by country, sourced from the Worldwide Health Organization.

I merge these data tables together to get international tourism revenues (as % of GDP) and measures of the speed and stringency of international travel restrictions for each country. Countries with no travel restriction data, no confirmed case data, and/or no international tourism (% of GDP) were dropped, leaving 140 countries. Because case data went into November, but travel restrictions data ended early October, I limited the timeframe to between 1 January 2020 and 1 October 2021. A small amount (0.97%) of travel restriction data was missing for the 140 countries over this timeframe, which I interpolated by country using forward fill under the assumption that the travel restriction data is more likely to have been updated when a change in travel restrictions occurred. 

To measure speed, I calculated the time between each country’s first case and when it first implemented each of the four levels of travel restrictions. To deal with cases where a country skipped a certain level of restriction, the measure is determined from the first day at a restriction at or higher than each level. For example, if a country went straight from level 1 to level 3 travel restrictions, the country’s time to level 2 would equal the time to level 3. To measure stringency, I calculated how many days the country had any travel restrictions (level 1-4), was quarantining or banning arrivals (level 2-4 controls), had any travel bans in place (level 3-4 controls), and had a total border closure (level 4 control). You can see descriptive statistics for each of these measures in Table 1. Only two countries never implemented any travel bans, while 25 countries never had a total border closure.

In [29]:
print('Table 1. Descriptive statistics.')
display(tourism_desc_df)

Table 1. Descriptive statistics.


,Tourism (% of GDP),Number days any travel controls (level 1-4),Number days quarantining or banning arrivals (level 2-4),Number days with travel bans (level 3-4),Number days with total border closure (level 4),Days from first case to travel controls,Days from first case to quarantining or banning arrivals (level 2+),Days from first case to any travel bans (level 3+),Days from first case to total border closure (level 4)
count,140.0,140.0,140.0,140.0,140.0,140.0,140.0,138.0,115.0
mean,5.7,594.7,485.6,360.3,152.5,-12.6,-1.3,8.3,36.1
std,6.9,52.0,145.8,172.9,139.4,42.8,41.7,54.4,83.3
min,0.1,416.0,76.0,0.0,0.0,-241.0,-241.0,-241.0,-230.0
25%,1.7,567.0,418.2,211.8,58.2,-37.2,-14.0,-1.0,9.0
50%,3.1,589.5,549.0,361.0,126.5,-4.0,3.5,6.5,16.0
75%,6.3,613.0,575.8,507.5,208.8,7.0,13.0,15.0,41.0
max,36.0,801.0,791.0,787.0,561.0,163.0,172.0,327.0,531.0


Overall, travel restrictions were implemented very quickly across the globe. Figure 1 shows a rapid implementation of travel bans (level 3 or 4 controls) in late March and early April followed by a gradual loosening of restrictions. By September 2020, nearly all countries had some form of international travel restrictions in place.

![Figure 1](images/travel-restrictions.pdf)

For an initial view on whether countries’ international tourism levels had an impact on the rollout of their international tourism categories, I map the travel restriction timeline by country in Figure 2. Because countries were hit at different times, the timeline is normalized by day of first case. The figure shows that there does not seem to be a pattern in the speed of timeline restrictions when normalized against day of first case: The rollout of restrictions hovers around the day of the first confirmed case (Day 0) regardless of tourism level.

![Figure 2](images/timeline-vs-tourism.pdf)

Finally, I plot each speed and stringency measure against international tourism levels in Figure 3. The top four scatterplots correspond to the speed measures, while the bottom four scatterplots correspond to the stringency measures. A few influential, extreme outliers were present in the speed measures, possibly due to anomalies in how quickly the first case appeared. To keep these from unduly suggesting a significant relationship, I trimmed any speed outliers with a z-score of over 3. None of the scatterplots had a significant p-value for the correlation between the measure international tourism (% of GDP): P-values for speed measures ranged from 0.12 to 0.5 and p-values for stringency measures ranged from 0.13 to 0.61. Similarly, the 95% confidence interval for the linear regression lines clearly straddle a slope of 0 for every measure except 'Days with controls', meaning we cannot be confident of either a positive or negative relationship between tourism as percent of GDP and our speed or stringency measures. We therefore cannot reject the null hypothesis that no relationship exists between pre-COVID international tourism levels and the implementation speed or stringency of travel restrictions. This is perhaps because governments largely prioritized public safety over more economic considerations, although further research would be needed to confirm this.

![Figure 3](images/restrictions_scatter.pdf)

# Part 2: Yelp reviews

## Introduction 

As cases rose and restrictions went into place, many US tourism businesses were forced to close or modify their businesses (OECD, 2020). Even after restrictions lessened, tourism businesses had to contend with increased customer anxieties. BCG’s COVID-19 Consumer Sentiment Survey from May 22-25, 2020 (N = 3,238) found that “worry” was a major constraint preventing respondents from doing tourism-related activities: More than 40% of respondents cited worry as the top reason for avoiding leisure travel, out-of-home entertainment, and eating at restaurants (BCG, 2020). To reduce anxiety, businesses began taking a number of measures, including increased sanitization, limited occupancy to maintain physical distancing, and requiring employees to wear personal protective equipment (McKinsey & Company, 2020).

Researchers have found noticeable increases in negative emotions reflected in online discourse during the pandemic. For example, one study found that posts in Reddit's largest hospitality-employee subreddit increased in anxiety, negative emotions, and anger during the pandemic (Park et al., 2020). Multiple other studies have found that restaurant reviews were also impacted by COVID-19. Cao et al. analyzed 3.1 million Yelp reviews using several machine learning techniques and found that more than 10% of reviews contained COVID-related keywords and that the prevalence of these keywords increased over the course of the pandemic (2021). Other studies have found that the lack of a COVID-safe dining environment has caused negative emotions towards restaurants and is associated with lower ratings (e.g. Kostromitina et al., 2021; Foroudi et al., 2021; Luo and Xu, 2021). Reviews are important to tourism businesses: They affect potential customers’ perceptions of businesses (Browning et al., 2013), customer demand (Phillips et al., 2017), and likelihood to book (Zhao et al., 2015), with implications on the popularity and success of restaurants (Anderson and Magruder, 2012). However, studies have largely focused on reviews within the dining industry rather than examining reviews within the broader tourism industry. This paper thus examines US Yelp reviews of tourism businesses according to RQ2:

> $RQ_2$: How did COVID-19 impact Yelp reviews in the tourism sector? 

## Results 

To investigate the impact of COVID-19 on reviews of US tourism businesses, I combine case data from the OxCOVID19 Database (Mahdi et al., 2021) with business and review data from Yelp, an online directory which enables users to find local businesses and publishes crowd-sourced reviews about these businesses. Yelp freely provides a selection of its data for research purposes (Yelp, n.d.-a). It does not allow researchers to access data via an API, but rather requires any potential users to fill out a form with their name and email that confirms consent with their terms of use. I therefore use the Mechanize library to fill out the form and download a zip file with the JSON data before reading the JSON files into DataFrames. Although the Yelp Open Dataset only provides a small fraction of the total data on the platform (see the section ‘Limitations and Conclusions’ below for a deeper discussion), using this dataset does have one benefit over the methods utilized by some other papers: It includes more historical data than web scraping methods that are only able to access the first page of reviews and therefore miss reviews from pre-pandemic and/or earlier in the pandemic (Kostromitina et al., 2021). 

I use the JSON files with review and business data, selecting data starting from January 1, 2020 to provide insights into review characteristics before COVID-19 disrupted the tourism industry. From the reviews table, I capture the stars (how highly a reviewer has rated the business, on an ordinal scale from 1 to 5), the business ID (used to connect the business data to the review), and the reviews themselves. From the business table, I capture location data and the business category. The category column provides a list of any number of categories that apply to a business; in total, 88,115 unique categories are captured within this field. I therefore use text filtering to assess whether each business belongs to one or more of the following tourism categories: hotel, transport, dining, and activity. I create separate Boolean categories for each rather than just one new column with category because businesses can fall under more than one category (e.g., a hotel with a restaurant would be tagged as both hotel and dining). Then I limit the business DataFrame to just tourism industries (going from 160,585 businesses to 77,985 businesses) before merging with the reviews data using inner join to drop any non-tourism reviews. Because countries differed in terms of restrictions and cases (Brookings, 2020), I only keep reviews from the US. In total, I end up with 450,547 reviews of US tourism businesses between 1 January 2020 and 28 January 2021.

To analyze how reviews that mention COVID-related words differed from other reviews, I identify key terms to check for. Given the complexity of language, no single list of will be exhaustive. However, to make sure I don’t miss any key COVID-related words, I run a Naïve Bayes model on a random sample of 50,000 reviews to identify the words that most predict whether a review was from before or after COVID. While the model is not very useful as a predictor (it performs slightly worse than just assuming all of the reviews are post-COVID), it does provide a useful list of the words that are most likely to show up in a post-COVID review, which I use as a guide when building out my own lists of COVID-related words to check for. Ultimately, I decide on 20 COVID-related words across four distinct categories: 
* words relating to the COVID crisis itself (COVID, coronavirus, pandemic, corona, virus)
* words relating to measures taken to reduce the spread of COVID (mask, distance, sanitize, gloves, sanitizer, quarantine, enforce, measure) 
* words relating to cancellations (refund, cancel, cancellation)
* words related to services that became much more common during the pandemic (curbside, deliver, contactless, delivery)

I then create columns with the number of words relating to each of the four COVID-19 word categories as well as the total number of COVID-19-related words in each review, and a Boolean column that indicates whether each review includes any COVID-19-related words.

Figure 4 plots review count and average review rating over time against daily US case counts. Total reviews dropped sharply in March, reflecting rising cases and new restrictions, including the required closure of certain sectors and businesses on March 16, 2020. Reviews increased somewhat between April and September as the US relaxed restrictions, before slightly dropping again as the third and largest peak in COVID cases hit the US in the fall and winter. Even as total reviews dropped, however, reviews that mentioned COVID-related terms increased. These reviews were more negative than reviews that didn’t include COVID-related terms, bringing down the average rating—albeit only slightly. This matches findings from other studies around reviews during COVID-19 (e.g. Kostromitina et al., 2021; Foroudi et al., 2021; Luo and Xu, 2021). Overall, however, average ratings did not vary much over time.

![Figure 4](images/yelp_reviews_all.pdf)

Figure 5 plots the share of reviews that mention different types of COVID-related words and the average rating of these reviews over time. Among the four categories of COVID-related words, words mentioning the crisis itself were the most common. All four categories increased in prevalence (measured as percent of total reviews) in March before decreasing at the tail end of the first peak in cases. Reviews that mentioned the COVID crisis itself or COVID-related measures increased again with the second peak in cases. Mentions of service changes and cancellations did not show a similar rise in mentions with the second or third peak, but did remain at an elevated level compared to the pre-COVID period. Average rating levels remained constant throughout the pandemic (with greater pre-pandemic variance likely due to the small number of reviews that mentioned these words). Reviews around cancellations had a significantly lower rating on average than reviews that included any of the other three categories of COVID-related words.

![Figure 5](images/yelp_reviews_covid.pdf)

The rating drop when COVID-related words are mentioned becomes even more pronounced the more times a review mentions these words. Figure 6 shows the impact of COVID anxieties on people's perceptions of tourism-related businesses. Not only is the average rating of reviews that mentioned COVID-related words lower, but the average rating of reviews decreases the more times they are mentioned. As found earlier, the average ratings do not seem to vary significantly over time, however. As Figure 7 shows, this trend holds true across the four different categories of COVID-related words, although the effect for crisis, measures, and service words are less extreme than words around cancellation: On average, even a single mention of ‘cancel,’ ‘cancellation,’ or ‘refund’ in a review is associated a negative star rating.

![Figure 6](images/heatmap_all.pdf)

![Figure 7](images/heatmap_cov_category.pdf)

To see whether the effect holds true across different types of tourism businesses, Figure 8 plots review counts and ratings by tourism business type. Although the different tourism businesses did differ in terms of the number of weekly reviews (the dining category has much more reviews on a weekly basis) and the average rating (the hotel category has notably lower average ratings), the COVID-19-related trends are consistent between categories. Each category saw reviews drop sharply in March and slowly increase in the summer before decreasing again in the winter, and reviews that mentioned COVID-19-related terms did drag down the average review rating for all four types of tourism businesses. Moreover, Figure 9 shows that the number of times a COVID-19-related word was used did have a negative impact on the rating for each type of tourism business on average, although the negative impact to hotel and activity reviews seems to be concentrated at the beginning of the pandemic.

![Figure 8](images/tourism_category.pdf)

![Figure 9](images/heatmap_tourism_category.pdf)

This analysis of US tourism reviews on Yelp shows that COVID-19 did have an impact on consumer perceptions of these businesses, with negative sentiment towards COVID-19-related topics spilling over into the text and star ratings of reviews. Overall, this analysis adds credence to findings from past studies around negative sentiments in reviews when COVID-19 safety measures aren’t followed (e.g. Kostromitina et al., 2021; Foroudi et al., 2021; Luo and Xu, 2021) and underlines the need for businesses to implement pandemic modifications for their customers.

# Limitations and conclusions

A few key limitations constrain the conclusions from Part 1. Firstly, the World Bank table in the OxCOVID19 Database doesn’t include full time-series data, which meant that 2018 tourism revenues had to be compared with 2019 GDP. While this still enabled a consistent method of comparing revenues, it could introduce some inaccuracies if, for example, a country’s currency changed drastically between 2018 or 2019. Although outside the bounds of this summative, this could be easily resolved in future research by using timewise-consistent data directly from the World Bank. Secondly, the Government Measures table did not include data on a few island countries with high levels of tourism (including Antigua and Barbuda, the Maldives, Saint Lucia, and others). Even for the countries that did have data within the Government Measures table, the manual methods used to create the Oxford COVID-19 Government Response Tracker introduce the possibility of entry errors or language misunderstandings, although the researchers actively sought to minimize these errors and ensure coding consistency through trainings and meetings as well as the use of a second coder to review every data point (Hale et al., 2021). Additionally, Part 1's analysis relies on the day of a country's first case to normalize speed. However, this could be impacted by confounding factors—e.g. how connected a country is to the rest of the world—that are not considered. Lastly, other factors might moderate the relationship between the size of the tourism industry and travel controls. To further explore the relationship, future research could account for lobbying power and/or the size of the largest travel players in each country, as this could impact how much the government would be willing to listen to the tourism sector.

The findings of Part 2 are also significantly caveated by limitations with the data and analysis. Firstly, the analysis is limited to the US, and the Yelp Open Dataset is limited to a small set of metropolitan areas (Yelp, n.d.-b). As different US states varied with respect to acceptance of COVID-19 safety measures (Fischer et al., 2021), future research could delve more deeply into whether these regional differences appeared in reviews. Secondly, the Yelp Open Dataset only includes reviews that Yelp recommended at the time of data collection (Yelp, n.d.-b). Thirdly,  Yelp’s user base skews younger, more affluent, and more highly educated than the general population (Bliss, 2021). Further research should examine whether the findings from this analysis hold for other regions and/or on alternative review sites. Lastly, the analysis relies on counts of specific COVID-19-related words rather than a more sophisticated language analysis.

# References 

Anderson, M., & Magruder, J. (2012). Learning from the Crowd: Regression Discontinuity Estimates of the Effects of an Online Review Database. *The Economic Journal, 122*(563). https://doi.org/10.1111/j.1468-0297.2012.02512.x

Aratani, L. (2021, April 30). Several countries are planning for international visitors. The U.S. travel industry fears being left behind. *Washington Post*. https://www.washingtonpost.com/transportation/2021/04/30/us-international-europe-travel/

BCG (2020). *COVID-19 Consumer Sentiment Snapshot 11*: Getting to the Other Side. https://www.bcg.com/publications/2020/covid-consumer-sentiment-survey-snapshot-6-02-20

Bliss, L. (2021, May 11). Where Covid’s Car-Free Streets Boosted Business. *Bloomberg CityLab*. https://www.bloomberg.com/news/articles/2021-05-11/the-business-case-for-car-free-streets

Brookings (2020, April 2). *The early days of a global pandemic: A timeline of COVID-19 spread and government interventions*. https://www.brookings.edu/2020/04/02/the-early-days-of-a-global-pandemic-a-timeline-of-covid-19-spread-and-government-interventions/

Brookings (2021). *The COVID-19 travel shock hit tourism-dependent economies hard*. https://www.brookings.edu/research/the-covid-19-travel-shock-hit-tourism-dependent-economies-hard/

Browning, V., So, K. K. F., & Sparks, B. (2013). The Influence of Online Reviews on Consumers’ Attributions of Service Quality and Control for Service Standards in Hotels. *Journal of Travel & Tourism Marketing, 30*(1–2). https://doi.org/10.1080/10548408.2013.750971

Cao, I., Liu, Z., Karamanolakis, G., Hsu, D., & Gravano, L. (2021). Quantifying the Effects of COVID-19 on Restaurant Reviews. *Proceedings of the Ninth International Workshop on Natural Language Processing for Social Media*. https://doi.org/10.18653/v1/2021.socialnlp-1.4

Deloitte (2020). *The future of hospitality: Uncovering opportunities to recover and thrive in the new normal*. https://www2.deloitte.com/content/dam/Deloitte/ca/Documents/consumer-industrial-products/ca-future-of-hospitality-pov-aoda-en.pdf

Fischer, C. B., Adrien, N., Silguero, J. J., Hopper, J. J., Chowdhury, A. I., & Werler, M. M. (2021). Mask adherence and rate of COVID-19 across the United States. *PLOS ONE, 16*(4), e0249891. https://doi.org/10.1371/journal.pone.0249891

Foroudi, P., H. Tabaghdehi, S. A., & Marvi, R. (2021). The gloom of the COVID-19 shock in the hospitality industry: A study of consumer risk perception and adaptive belief in the dark cloud of a pandemic. *International Journal of Hospitality Management, 92*. https://doi.org/10.1016/j.ijhm.2020.102717

Gössling, S., Scott, D., & Hall, C. M. (2021). Pandemics, tourism and global change: a rapid assessment of COVID-19. *Journal of Sustainable Tourism, 29*(1). https://doi.org/10.1080/09669582.2020.1758708

Hale, T., Webster, S., Petherick, A., Phillips, T., & Kira, B. (2020). Oxford COVID-19 Government Response Tracker, Blavatnik School of Government.

Hale, T., Angrist, N., Goldszmidt, R., Kira, B., Petherick, A., Phillips, T., Webster, S., Cameron-Blake, E., Hallas, L., Majumdar, S., & Tatlow, H. (2021). A global panel database of pandemic policies (Oxford COVID-19 Government Response Tracker). *Nature Human Behaviour, 5*(4), 529–538. https://doi.org/10.1038/s41562-021-01079-8

Hall, C. M., Scott, D., & Gössling, S. (2020). Pandemics, transformations and tourism: be careful what you wish for. *Tourism Geographies, 22*(3). https://doi.org/10.1080/14616688.2020.1759131

Kostromitina, M., Keller, D., Cavusoglu, M., & Beloin, K. (2021). “His lack of a mask ruined everything.” Restaurant customer satisfaction during the COVID-19 outbreak: An analysis of Yelp review texts and star-ratings. *International Journal of Hospitality Management, 98*. https://doi.org/10.1016/j.ijhm.2021.103048

Luo, Y., & Xu, X. (2021). Comparative study of deep learning models for analyzing online restaurant reviews in the era of the COVID-19 pandemic. *International Journal of Hospitality Management, 94*. https://doi.org/10.1016/j.ijhm.2020.102849

Mahdi, A., Błaszczyk, P., Dłotko, P., Salvi, D., Chan, T.-S., Harvey, J., Gurnari, D., Wu, Y., Farhat, A., Hellmer, N., Zarebski, A., Hogan, B., & Tarassenko, L. (2021). OxCOVID19 Database, a multimodal data repository for better understanding the global impact of COVID-19. *Scientific Reports, 11*(1). https://doi.org/10.1038/s41598-021-88481-4

McKinsey & Company (2020). *Eating out(side): Restaurant dining in the next normal*. https://www.mckinsey.com/~/media/McKinsey/Industries/Retail/Our%20Insights/Eating%20out%20side%20Restaurant%20dining%20in%20the%20next%20normal/Eating-outside-Restaurant-dining-in-the-next-normal.pdf

OECD (2020). *Tourism Policy Responses to the coronavirus (COVID-19)*. https://www.oecd.org/coronavirus/policy-responses/tourism-policy-responses-to-the-coronavirus-covid-19-6466aa20/

Park, E., Kim, W.-H., & Kim, S.-B. (2020). Tracking tourism and hospitality employees’ real-time perceptions and emotions in an online community during the COVID-19 pandemic. *Current Issues in Tourism*. https://doi.org/10.1080/13683500.2020.1823336

Phillips, P., Barnes, S., Zigan, K., & Schegg, R. (2017). Understanding the Impact of Online Reviews on Hotel Performance. *Journal of Travel Research, 56*(2). https://doi.org/10.1177/0047287516636481

Seyfi, S., Hall, C. M., & Shabani, B. (2020). COVID-19 and international travel restrictions: the geopolitics of health and tourism. *Tourism Geographies*, 1–17. https://doi.org/10.1080/14616688.2020.1833972

UNCTAD (2021). *COVID-19 and Tourism – An Update: Assessing the economic consequences*. https://unctad.org/system/files/official-document/ditcinf2021d3_en_0.pdf

UNWTO (2020). *100% of Global Destinations Now Have COVID-19 Travel Restrictions*. https://www.unwto.org/news/covid-19-travel-restrictions

UNWTO (2021a). *Secretary-General’s Policy Brief on Tourism and COVID-19: Unprecedented Economic Impacts*. https://www.unwto.org/tourism-and-covid-19-unprecedented-economic-impacts

UNWTO (2021b). *2020: Worst Year in Tourism History with 1 Billion Fewer International Arrivals*. https://www.unwto.org/news/2020-worst-year-in-tourism-history-with-1-billion-fewer-international-arrivals

World Bank (2018). *International tourism, receipts (current USD)*. Retrieved from https://data.worldbank.org/indicator/ST.INT.RCPT.CD

World Bank (2019). *GDP (current USD)*. Retrieved from https://data.worldbank.org/indicator/NY.GDP.MKTP.CD

World Travel & Tourism Council (2021). *Travel & Tourism: Economic Impact 2021*. https://wttc.org/Portals/0/Documents/EIR/EIR2021%20Global%20Infographic.pdf?ver=2021-04-06-170951-897

Yelp. (n.d.-a). *Yelp Open Dataset*. Retrieved December 2, 2021, from https://www.yelp.com/dataset

Yelp. (n.d.-b). *Yelp Dataset: Frequently Asked Questions*. Retrieved December 2, 2021, from https://www.yelp.com/dataset/documentation/faq

Zhao, X. (Roy), Wang, L., Guo, X., & Law, R. (2015). The influence of online reviews to online hotel booking intentions. *International Journal of Contemporary Hospitality Management, 27*(6). https://doi.org/10.1108/IJCHM-12-2013-0542

In [31]:
# Word count
# Drawing on code from:
# https://gist.github.com/agounaris/5da16c233ce480e75ab95980831f459e 

import io
from IPython.nbformat import current
from pathlib import Path

with io.open(Path().cwd() / 'FSDS_summative_1058119.ipynb', 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')

word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown" and cell['source'][:12] != "# References":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print(f"The word count, excluding references, is {word_count}")

The word count, excluding references, is 3490
